In [1]:
from sqlalchemy import create_engine, select
from sqlalchemy.orm import clear_mappers, sessionmaker

from deploy.adapters import orm
from deploy.bootstrap import bootstrap
from deploy.domain.model import Service
from deploy.service_layer import unit_of_work

# Setup

In [2]:
engine = create_engine("postgresql:///deploy")
orm.metadata_obj.create_all(engine)
orm.start_mappers()
connection = engine.connect()
transaction = connection.begin()
session = sessionmaker()(bind=connection)

start mappers


# Doing stuff that gets rolled back afterwards

In [3]:
service = Service(name="asdf", data={"asdf": "bsdf"})
session.add(service)

In [5]:
[service_from_db] = session.execute(select(Service).where(Service.name == "asdf")).one()
print(service_from_db.id)

2


# Doing stuff with Unit of Work

Fail before session proxy, should work afterwards..

In [3]:
class SessionProxy:
    def __init__(self, session):
        self.session = session

    def __getattr__(self, attr):
        """Proxy all unknown attributes."""
        print("attr: ", attr)
        return getattr(self.session, attr)

    def close(self):
        """Disable close because __exit__ of SqlAlchemyUnitOfWork calls it :/"""
        print("called disabled close..")
        pass

    def rollback(self):
        """Disable rollback because __exit__ of AbstractUnitOfWork calls it :/"""
        print("called disabled rollback..")

In [4]:
def session_factory():
    return session
    # return SessionProxy(session)


# uow = unit_of_work.SqlAlchemyUnitOfWork(session_factory)
uow = unit_of_work.TestableSqlAlchemyUnitOfWork(session_factory)
bus = bootstrap(start_orm=False, uow=uow)

In [5]:
service = Service(name="fastdeploytest", data={"foo": "bar"})
with bus.uow as uow:
    uow.services.add(service)
    # uow.commit()
    [service_from_db] = uow.services.get(service.name)

In [6]:
service_from_db.id

12

In [7]:
with bus.uow as uow:
    [service_from_db] = uow.services.get(service.name)
print(service_from_db.id)

12


## Trying to Find out if it's Always Tuples?

In [12]:
with bus.uow as uow:
    print(uow.services.session.execute(select(Service).where(Service.name == service.name)).one())

(<deploy.domain.model.Service object at 0x104df0610>,)


In [10]:
with bus.uow as uow:
    print(uow.services.session.execute(select(Service).where(Service.name == service.name)).one_or_none())

(<deploy.domain.model.Service object at 0x104df0610>,)


In [11]:
with bus.uow as uow:
    print(uow.services.session.execute(select(Service).where(Service.name == service.name)).fetchone())

(<deploy.domain.model.Service object at 0x104df0610>,)


In [9]:
with bus.uow as uow:
    print(uow.services.session.execute(select(Service).where(Service.name == service.name)).first())

(<deploy.domain.model.Service object at 0x104df0610>,)


In [14]:
with bus.uow as uow:
    print(uow.services.session.execute(select(Service)).all())

[(<deploy.domain.model.Service object at 0x104df0610>,)]


## Oh Wow!

In [15]:
with bus.uow as uow:
    print(uow.services.session.query(Service).filter_by(name=service.name).first())

# Teardown

In [9]:
session.close()
transaction.rollback()
connection.close()

## Removes all Tables

In [8]:
orm.metadata_obj.drop_all(engine)
clear_mappers()